<a href="https://colab.research.google.com/github/VINY1958/polars/blob/main/polars_gpu_engine_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.youtube.com/watch?v=8GoBlwgbirE


https://github.com/rapidsai-community/showcase/blob/main/accelerated_data_processing_examples/polars_gpu_engine_demo.ipynb?utm_source=influencer&utm_medium=social&utm_campaign=MS

## Introduction to the Polars GPU Engine

Polars is a popular single machine DataFrame library powered by an OLAP Query Engine. Beginning in the v1.3 release, Polars can now leverage NVIDIA GPUs for even higher performance through its GPU engine (powered by RAPIDS cuDF).

Designed to make processing 10-100+ GBs of data feel interactive with just a single GPU, this new engine is built directly into the Polars Lazy API – just pass <font color="#76B900">**engine="gpu"**</font> to the `collect` operation.

The GPU engine fully utilizes the Polars optimizer to ensure efficient execution and minimal memory usage, is compatible with the ecosystem of tools built for Polars, and has graceful CPU fallback for unsupported queries.

This notebook is a short introduction to the Polars GPU engine, powered by cuDF.

Polars es una popular biblioteca de datos de datos de una sola máquina alimentada por un motor OLAP Query. A partir de la liberación de V1.3, los polares ahora pueden aprovechar las GPU NVIDIA para un rendimiento aún mayor a través de su motor GPU (alimentado por Rapids CUDF).

Diseñado para hacer que el procesamiento de más de 10-100 GB de datos se sienta interactivo con una sola GPU, este nuevo motor se basa directamente en la API Polars Lazy-Just Pass Engine = "GPU" para la operación de recolección.

El motor GPU utiliza completamente el optimizador Polars para garantizar una ejecución eficiente y un uso mínimo de memoria, es compatible con el ecosistema de herramientas creadas para polares, y tiene un elegante retroceso de la CPU para consultas no compatibles.

Este cuaderno es una breve introducción al motor GPU Polars, alimentado por CUDF.


In [1]:
!nvidia-smi | head

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
!lscpu | grep -E "Model|Socket"

Model name:                           Intel(R) Xeon(R) CPU @ 2.20GHz
Model:                                79
Socket(s):                            1


# Setup

## Installing the Polars GPU Engine

Install Polars with GPU Engine by using a feature flag in the standard pip install command  `pip install polars[gpu] --extra-index-url=https://pypi.nvidia.com`.

Since this notebook may be used in environments with an existing Polars installation (e.g., Google Colab), we'll add the `-U` flag to upgrade Polars if it's already present.

Instale los polares con motor GPU utilizando un indicador de características en el comando estándar de instalación PIP PIP Instale POLARS [GPU] --extra-Index-URL = https: //pypi.nvidia.com.

Dado que este cuaderno puede usarse en entornos con una instalación de Polars existente (por ejemplo, Google Colab), agregaremos el indicador -U para actualizar los polares si ya está presente.

In [3]:
!pip install -U polars[gpu] --extra-index-url=https://pypi.nvidia.com

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


# Install Other Important Dependencies
To use the built-in data visualization capabilities of Polars, you'll need to install a few additional dependencies.  We'll also install pynvml to help us determine which dataset size to use.

Para usar las capacidades de visualización de datos incorporadas de los polares, deberá instalar algunas dependencias adicionales. También instalaremos pynvml para ayudarnos a determinar qué tamaño de conjunto de datos usar.

In [4]:
!pip install hvplot jupyter_bokeh holoviews==1.19 pynvml

## Download the Data

We'll be working with a roughly 22GB dataset of [simulated financial transactions from Kaggle](https://www.kaggle.com/datasets/conorsully1/simulated-transactions) by default. If you're running this notebook on Google Colab using the T4 GPU in the Colab free tier, we'll download a smaller version of this dataset (about 20% of the size) to fit on the relatively weaker CPU and GPU.

We're downloading a copy of this dataset from a GCS bucket hosted by NVIDIA to provide faster download speeds. We'll start by downloading the data. This should take about 30 seconds.

We'll be working with a roughly 22GB dataset of simulated financial transactions from Kaggle by default. If you're running this notebook on Google Colab using the T4 GPU in the Colab free tier, we'll download a smaller version of this dataset (about 20% of the size) to fit on the relatively weaker CPU and GPU.

We're downloading a copy of this dataset from a GCS bucket hosted by NVIDIA to provide faster download speeds. We'll start by downloading the data. This should take about 30 seconds.

In [5]:
!pip install pynvml

In [6]:
!apt-get update
!apt-get install -y --no-install-recommends cuda-drivers nvidia-driver-470
!apt-get install -y nvidia-utils-470 # or the version you need
!pip install pynvml

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 2s (141 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to prov

In [7]:
!apt-get update
!apt-get install -y --no-install-recommends cuda-drivers nvidia-driver-470
!apt-get install -y nvidia-utils-470 # or the version you need
!pip install pynvml

import pynvml

try:
    # Try initializing NVML
    pynvml.nvmlInit()
    # If initialization is successful, get GPU information
    device_name = pynvml.nvmlDeviceGetName(pynvml.nvmlDeviceGetHandleByIndex(0)).decode('utf-8')  # Decode to string
    mem_info = pynvml.nvmlDeviceGetMemoryInfo(pynvml.nvmlDeviceGetHandleByIndex(0))
    mem = mem_info.total / 1e9

    # Download the appropriate parquet file based on GPU memory
    if mem < 24:
        !wget https://storage.googleapis.com/rapidsai/polars-demo/transactions-t4-20.parquet -O transactions.parquet
    else:
        !wget https://storage.googleapis.com/rapidsai/polars-demo/transactions.parquet -O transactions.parquet

except pynvml.NVMLError_DriverNotLoaded:
    # If driver is not loaded, print an informative message and download the smaller file.
    print("NVIDIA driver is not loaded. Downloading smaller parquet file.")
    !wget https://storage.googleapis.com/rapidsai/polars-demo/transactions-t4-20.parquet -O transactions.parquet

# Download the rainfall data
!wget https://storage.googleapis.com/rapidsai/polars-demo/rainfall_data_2010_2020.csv

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pack

## Data License and Terms
As this dataset originates from Kaggle, it's governed by a Kaggle dataset-specific license and terms of use.

> ### CC0 1.0 Universal
> **No Copyright**

>The person who associated a work with this deed has dedicated the work to the public domain by waiving all of his or her rights to the work worldwide under copyright law, including all related and neighboring rights, to the extent allowed by law.
You can copy, modify, distribute and perform the work, even for commercial purposes, all without asking permission. See Other Information below.

> **Other Information**

>In no way are the patent or trademark rights of any person affected by CC0, nor are the rights that other persons may have in the work or in how the work is used, such as publicity or privacy rights.
Unless expressly stated otherwise, the person who associated a work with this deed makes no warranties about the work, and disclaims liability for all uses of the work, to the fullest extent permitted by applicable law.
When using or citing the work, you should not imply endorsement by the author or the affirmer.


Licencia de datos y términos
Como este conjunto de datos se origina en Kaggle, se rige por una licencia y términos de uso específico de conjunto de datos de Kaggle.

CC0 1.0 Universal
Sin derechos de autor

La persona que asoció un trabajo con esta escritura ha dedicado el trabajo al dominio público al renunciar a todos sus derechos sobre el trabajo de todo el mundo bajo la ley de derechos de autor, incluidos todos los derechos relacionados y vecinos, en la medida permitida por la ley. Puede copiar, modificar, distribuir y realizar el trabajo, incluso para fines comerciales, todo sin pedir permiso. Vea otra información a continuación.

Otra información

De ninguna manera son los derechos de patente o marca registrada de ninguna persona afectada por CC0, ni los derechos que otras personas pueden tener en el trabajo o en cómo se usa el trabajo, como la publicidad o los derechos de privacidad. A menos que se indique expresamente lo contrario, la persona que asoció un trabajo con esta escritura no ofrece garantías sobre el trabajo y renuncia a la responsabilidad por todos los usos del trabajo, en la mayor medida permitida por la ley aplicable. Al usar o citar el trabajo, no debe implicar el respaldo del autor o el afirmador.


# Getting Started

With this dataset, we expect to see some meaningful performance benefits using NVIDIA GPUs for computationally heavy queries and limited benefits for basic queries that are primarily bottlenecked by reading data.

To begin, let's use Polars to read the parquet file and look at the schema and first few rows.

Con este conjunto de datos, esperamos ver algunos beneficios de rendimiento significativos utilizando las GPU de NVIDIA para consultas computacionalmente pesadas y beneficios limitados para consultas básicas que son principalmente cuellos de botella mediante la lectura de datos.

Para comenzar, usemos polares para leer el archivo de parquet y ver el esquema y las primeras filas.

In [8]:
import polars as pl
import hvplot.polars # Using the hvplot as the plotting backend
from polars.testing import assert_frame_equal

In [9]:
pl.__version__ # Make sure the version is >= `1.6.0`; If not restart the session.

'1.12.0'

In [10]:
transactions = pl.scan_parquet("transactions.parquet")

In [11]:
import os
import polars as pl

# Get the current working directory
current_directory = os.getcwd()
print(f"Current working directory: {current_directory}")

# Construct the full path to the Parquet file
file_path = os.path.join(current_directory, "transactions.parquet")

# Check if the file exists
if os.path.exists(file_path):
    print(f"File found at: {file_path}")
    transactions = pl.scan_parquet(file_path) # Change this line
    transactions.collect_schema()
else:
    print(f"File not found at: {file_path}")
    print("Please make sure the 'transactions.parquet' file is in the current directory or provide the correct path.")

Current working directory: /content
File found at: /content/transactions.parquet


In [12]:
transactions.collect()

CUST_ID,START_DATE,END_DATE,TRANS_ID,DATE,YEAR,MONTH,DAY,EXP_TYPE,AMOUNT
str,date,date,str,date,i64,i64,i64,str,f64
"""CI6XLYUMQK""",2015-05-01,null,"""T8I9ZB5A6X90UG8""",2015-09-11,2015,9,11,"""Motor/Travel""",20.27
"""CI6XLYUMQK""",2015-05-01,null,"""TZ4JSLS7SC7FO9H""",2017-02-08,2017,2,8,"""Motor/Travel""",12.85
"""CI6XLYUMQK""",2015-05-01,null,"""TTUKRDDJ6B6F42H""",2015-08-01,2015,8,1,"""Housing""",383.8
"""CI6XLYUMQK""",2015-05-01,null,"""TDUHFRUKGPPI6HD""",2019-03-16,2019,3,16,"""Entertainment""",5.72
"""CI6XLYUMQK""",2015-05-01,null,"""T0JBZHBMSVRFMMD""",2015-05-15,2015,5,15,"""Entertainment""",11.06
…,…,…,…,…,…,…,…,…,…
"""CDM6EPDW8A""",2017-07-01,null,"""TBEPJBHH7QIWUQK""",2019-03-09,2019,3,9,"""Entertainment""",53.46
"""CDM6EPDW8A""",2017-07-01,null,"""T6FNDF7HXYDQGE5""",2019-09-24,2019,9,24,"""Entertainment""",62.27
"""CDM6EPDW8A""",2017-07-01,null,"""TVFAEMNFM83OCB9""",2020-05-11,2020,5,11,"""Motor/Travel""",2005.97


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
transactions.head(5).collect()

CUST_ID,START_DATE,END_DATE,TRANS_ID,DATE,YEAR,MONTH,DAY,EXP_TYPE,AMOUNT
str,date,date,str,date,i64,i64,i64,str,f64
"""CI6XLYUMQK""",2015-05-01,null,"""T8I9ZB5A6X90UG8""",2015-09-11,2015,9,11,"""Motor/Travel""",20.27
"""CI6XLYUMQK""",2015-05-01,null,"""TZ4JSLS7SC7FO9H""",2017-02-08,2017,2,8,"""Motor/Travel""",12.85
"""CI6XLYUMQK""",2015-05-01,null,"""TTUKRDDJ6B6F42H""",2015-08-01,2015,8,1,"""Housing""",383.8
"""CI6XLYUMQK""",2015-05-01,null,"""TDUHFRUKGPPI6HD""",2019-03-16,2019,3,16,"""Entertainment""",5.72
"""CI6XLYUMQK""",2015-05-01,null,"""T0JBZHBMSVRFMMD""",2015-05-15,2015,5,15,"""Entertainment""",11.06


In [15]:
###data = pl.scan_csv("transactions.csv")

In [16]:
####data.collect_schema()
##data.sink_parquet("transactions.parquet") achico el archivo

In [17]:
transactions.select(pl.col('CUST_ID').n_unique()).collect(engine='gpu')

CUST_ID
u32
14282


In [18]:
transactions.schema

<ipython-input-18-e18b1eccf6b4>:1: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  transactions.schema


Schema([('CUST_ID', String),
        ('START_DATE', Date),
        ('END_DATE', Date),
        ('TRANS_ID', String),
        ('DATE', Date),
        ('YEAR', Int64),
        ('MONTH', Int64),
        ('DAY', Int64),
        ('EXP_TYPE', String),
        ('AMOUNT', Float64)])

In [19]:
transactions.collect(engine='gpu')

CUST_ID,START_DATE,END_DATE,TRANS_ID,DATE,YEAR,MONTH,DAY,EXP_TYPE,AMOUNT
str,date,date,str,date,i64,i64,i64,str,f64
"""CI6XLYUMQK""",2015-05-01,null,"""T8I9ZB5A6X90UG8""",2015-09-11,2015,9,11,"""Motor/Travel""",20.27
"""CI6XLYUMQK""",2015-05-01,null,"""TZ4JSLS7SC7FO9H""",2017-02-08,2017,2,8,"""Motor/Travel""",12.85
"""CI6XLYUMQK""",2015-05-01,null,"""TTUKRDDJ6B6F42H""",2015-08-01,2015,8,1,"""Housing""",383.8
"""CI6XLYUMQK""",2015-05-01,null,"""TDUHFRUKGPPI6HD""",2019-03-16,2019,3,16,"""Entertainment""",5.72
"""CI6XLYUMQK""",2015-05-01,null,"""T0JBZHBMSVRFMMD""",2015-05-15,2015,5,15,"""Entertainment""",11.06
…,…,…,…,…,…,…,…,…,…
"""CDM6EPDW8A""",2017-07-01,null,"""TBEPJBHH7QIWUQK""",2019-03-09,2019,3,9,"""Entertainment""",53.46
"""CDM6EPDW8A""",2017-07-01,null,"""T6FNDF7HXYDQGE5""",2019-09-24,2019,9,24,"""Entertainment""",62.27
"""CDM6EPDW8A""",2017-07-01,null,"""TVFAEMNFM83OCB9""",2020-05-11,2020,5,11,"""Motor/Travel""",2005.97


In [20]:
transactions.filter(
    (pl.col("YEAR") == 2020) &
    (pl.col("EXP_TYPE") == "Housing")
).select(
    pl.sum("AMOUNT").cast(pl.Int64)
).collect(engine="gpu")

AMOUNT
i64
55734527


In [ ]:
transactions.group_by(
    "EXP_TYPE"
).agg(
    pl.sum("AMOUNT")
).sort(
    by="AMOUNT", descending=True
).collect(engine="gpu")

In [ ]:
transactions.filter(
    pl.col("EXP_TYPE").is_in(["Motor/Travel", "Housing"])
).group_by(
    "EXP_TYPE"
).agg(
    pl.sum("AMOUNT")
).sort(
    by="AMOUNT", descending=True
).collect(engine="gpu")
transactions.filter(
    pl.col("EXP_TYPE").is_in(["Motor/Travel", "Housing"])
).group_by(
    "EXP_TYPE"
).agg(
    pl.sum("AMOUNT")
).sort(
    by="AMOUNT", descending=True
).collect(engine="gpu")
transactions.group_by(
    "EXP_TYPE"
).agg(
    [
        pl.sum("AMOUNT"),
        pl.mean("AMOUNT").alias("AVERAGE"),
        pl.count("AMOUNT").alias("COUNT") # This line was incorrectly indented. Moved inside agg()
    ]
).sort(
    by="AMOUNT", descending=True
).collect(engine="gpu")
# The following lines were causing the error. Removed them.
#        pl.count("AMOUNT").alias("COUNT")
#).sort(
#    by="AMOUNT", descending=True

In [ ]:
transactions.filter(
    pl.col("EXP_TYPE").is_in (["Motor/Travel", "Housing"])
).group_by(
    "EXP_TYPE"
).agg(
    pl.sum("AMOUNT")
).sort(
    by="AMOUNT", descending=True
).collect(engine="gpu")
transactions.filter(
    pl.col("EXP_TYPE").is_in(["Motor/Travel", "Housing"])
).group_by(
    "EXP_TYPE"
).agg(
    pl.sum("AMOUNT")
).sort(
    by="AMOUNT", descending=True
).collect(engine="gpu")
transactions.group_by(
    "EXP_TYPE"
).agg(
    [
        pl.sum("AMOUNT"),
        pl.mean("AMOUNT").alias("AVERAGE"),
        pl.count("AMOUNT").alias("COUNT")
    ]
).sort(
    by="AMOUNT", descending=True
).collect(engine="gpu")
# The error was caused by these lines. They were outside the agg() function and indented unnecessarily.
# I have removed them to fix the error.
# pl.count("AMOUNT").alias("COUNT") # This line was incorrectly placed outside the agg() function.
# ).sort(
#     by="AMOUNT", descending=True
# ).collect(engine="gpu")

In [ ]:


transactions.filter(
    pl.col("EXP_TYPE").is_in (["Motor/Travel", "Housing"])
).group_by(
    "EXP_TYPE"
).agg(
    pl.sum("AMOUNT")
).sort(
    by="AMOUNT", descending=True
).collect(engine="gpu")
transactions.filter(
    pl.col("EXP_TYPE").is_in(["Motor/Travel", "Housing"])
).group_by(
    "EXP_TYPE"
).agg(
    pl.sum("AMOUNT")
).sort(
    by="AMOUNT", descending=True
).collect(engine="gpu")
transactions.group_by(
    "EXP_TYPE"
).agg(
    [
        pl.sum("AMOUNT"),
        pl.mean("AMOUNT").alias("AVERAGE"),
        pl.count("AMOUNT").alias("COUNT")
    ] # The extra pl.count line was removed from here. It was causing the indentation error as it was outside the agg() function
).sort(
    by="AMOUNT", descending=True
).collect(engine="gpu")
# The following lines were causing the error due to incorrect indentation
#        pl.count("AMOUNT").alias("COUNT")
#).sort(
#    by="AMOUNT", descending=True
#).collect(engine="gpu")

In [ ]:
transactions.group_by(
    "EXP_TYPE"
).agg(
    pl.sum("AMOUNT")
).sort(
    by="AMOUNT", descending=True
).collect(engine="gpu")
transactions.group_by(
    "EXP_TYPE"
).agg(
    [
        pl.sum("AMOUNT"),
        pl.mean("AMOUNT").alias("AVERAGE"),
        pl.count("AMOUNT").alias("COUNT")
    ]
).sort(
    by="AMOUNT", descending=True
).collect(engine="gpu").plot.bar(x="EXP_TYPE", y="AVERAGE")
transactions.group_by(
    "EXP_TYPE"
).agg(
    pl.sum("AMOUNT")
).sort(
    by="AMOUNT", descending=True
).collect(engine="gpu")
transactions.group_by(
    "EXP_TYPE"
).agg(
    [
        pl.sum("AMOUNT"),
        pl.mean("AMOUNT").alias("AVERAGE"),
        pl.count("AMOUNT").alias("COUNT")
    ]
).sort(
    by="AMOUNT", descending=True
).collect(engine="gpu").plot.bar(x="EXP_TYPE", y="AVERAGE")
transactions.group_by(
    "EXP_TYPE"
).agg(
    pl.sum("AMOUNT")
).sort(
    by="AMOUNT", descending=True
).collect(engine="gpu")
transactions.group_by(
    "EXP_TYPE"
).agg(
    [
        pl.sum("AMOUNT"),
        pl.mean("AMOUNT").alias("AVERAGE"),
        pl.count("AMOUNT").alias("COUNT")
    ]
).sort(
    by="AMOUNT", descending=True
).collect(engine="gpu").plot.bar(x="EXP_TYPE", y="AVERAGE")
).agg(
    [
        pl.sum("AMOUNT"),
        pl.mean("AMOUNT").alias("AVERAGE"),
        pl.count("AMOUNT").alias("COUNT")
    ]
).sort(
    by="AMOUNT", descending=True
).collect(engine="gpu").plot.bar(x="EXP_TYPE", y="AVERAGE")
#).sort( #removed this line, it was causing a syntax error
#    by="AMOUNT", descending=True
#).collect(engine="gpu").plot.bar(x="EXP_TYPE", y="AVERAGE")

In [ ]:
import altair as alt
##alt.data_transformers.enable("vegafusion")

In [ ]:
!pip install --upgrade altair>=5.4.0

In [ ]:
import polars as pl
import altair as alt # Import altair

# ... your code ...

transactions.head(100).collect(engine="gpu").plot.scatter(
    x="AMOUNT", y="DATE", color="EXP_TYPE"
).properties(width=750)

In [ ]:
transactions.head(1000).collect(engine="gpu").plot.scatter(
    x="AMOUNT", y="DATE", color="EXP_TYPE"
).properties(width=750)

In [ ]:
transactions.head(1000).collect(engine="gpu").plot.scatter(
    x="AMOUNT", y="DATE"
).properties(width=750).mark_point(size=250, strokeWidth=1)

In [ ]:
!pip install vegafusion

In [ ]:
!pip install vegafusion # Ensure vegafusion is installed
import altair as alt
# Import vegafusion to potentially register the entry point
import vegafusion
###alt.data_transformers.enable("vegafusion")

In [ ]:
import altair as alt
alt.data_transformers.enable("vegafusion")

In [ ]:
transactions.head(100).collect(engine="gpu").plot.scatter(
    x="AMOUNT", y="DATE", color="EXP_TYPE", tooltip=[
        "TRANSACTION_ID","CUST_ID", "DATE", "EXP_TYPE"
    ]
).properties(
    width=750, title="Expense Amountsand Type Over Time"
).mark_circle(
    size=250, strokeWidth=1
)

In [ ]:
import plotly.express as px

placa de hv
Si importa hvplot.polars, registra un método HVPLOT que puede usar para crear gráficos interactivos usando HVPLOT.

In [ ]:
!pip install hvplot

In [ ]:
import hvplot.polars

In [ ]:
transactions.head(1000000).collect(engine="gpu").plot.scatter(
    x="AMOUNT", y="DATE", color="EXP_TYPE", tooltip=[
        "TRANSACTION_ID","CUST_ID", "DATE", "EXP_TYPE"
    ]).properties(
    width=750, title="Expense Amountsand Type Over Time"
).mark_circle(
    size=250, strokeWidth=1
).properties(
    width=750, title="Expense Amountsand Type Over Time"
).mark_circle(
    size=250, strokeWidth=1
)

This dataset has a mix of data types (strings, dates, integers, and floats), some missing values, and it looks like the dates were actually parsed as strings. We'll need to handle that during some of our analysis.

Este conjunto de datos tiene una combinación de tipos de datos (cadenas, fechas, enteros y flotadores), algunos valores faltantes, y parece que las fechas en realidad se analizaron como cadenas. Tendremos que manejar eso durante algunos de nuestro análisis.

# Our first GPU operation: Total aggregate transaction amount

With Polars, to calculate the total spending in of all transactions we simply `sum` the `AMOUNT` column:

Con polares, para calcular el gasto total en todas las transacciones, simplemente sumamos la columna de monto:

In [ ]:
transactions.select(pl.col("AMOUNT").sum()).collect()

Looks like we're handling a high total transaction volume. Let's run the same query on the GPU.

To use a GPU, we just need to tell Polars to use the GPU engine when we call `collect` by passing `engine="gpu"` as a parameter.


```python
transactions.select(pl.col("AMOUNT").sum()).collect(engine="gpu")
```

The default configuration is likely the right choice for most workloads, but sometimes we want more control. We can provide more detailed configuration options (such as which device to run on and a variety of other options) by passing a Polars `GPUEngine` object to the `engine` parameter instead.


In this notebook, we'll use `pl.GPUEngine`. The default configuration has transparent CPU fallback for unsupported operations, so if we execute an unsupported query we don't get an error. To prove we're running on the GPU, we'll pass a configured engine object that will raise an error if we can't run the query.

_If you're running with [jupyterlab-nvdashboard](https://developer.nvidia.com/blog/maximize-gpu-performance-with-near-real-time-usage-stats-on-nvdashboard-v0-10/) you should see the GPU Memory and Utilization tick up._

🤓

996 / 5.000
Parece que estamos manejando un volumen de transacción total alto. Ejecutemos la misma consulta en la GPU.

Para usar una GPU, solo necesitamos indicarle a los polares que usen el motor GPU cuando llamamos a recolectar el motor = "GPU" como parámetro.

transacciones.select (pl.col ("cantidad"). Sum ()). Collect (Engine = "GPU")
La configuración predeterminada es probablemente la opción correcta para la mayoría de las cargas de trabajo, pero a veces queremos más control. Podemos proporcionar opciones de configuración más detalladas (como qué dispositivo ejecutar y una variedad de otras opciones) pasando un objeto Polar GpuEngine al parámetro del motor.

En este cuaderno, usaremos pl.gpuEngine. La configuración predeterminada tiene alternativa de CPU transparente para operaciones no compatibles, por lo que si ejecutamos una consulta no compatible, no recibimos un error. Para demostrar que nos estamos ejecutando en la GPU, pasaremos un objeto de motor configurado que aumentará un error si no podemos ejecutar la consulta.

Si estás ejecutando con JupyterLab-NvDashboard, deberías ver la memoria de GPU y la utilización marcada.
Enviar comentarios


In [ ]:
gpu_engine = pl.GPUEngine(
    device=0, # This is the default
    raise_on_fail=True, # Fail loudly if we can't run on the GPU.
)

The very first collection on the GPU will take a couple of seconds. The GPU engine is lazy-loaded so that even if the necessary packages are installed, Polars' fast import times are not affected. Consequently, when we trigger GPU execution for the first time, we load a number of additional packages, and initialize GPU-specific data structures and contexts.

La primera colección en la GPU tomará un par de segundos. El motor GPU está cargado de perezoso para que incluso si se instalan los paquetes necesarios, los rápidos tiempos de importación de Polars no se ven afectados. En consecuencia, cuando activamos la ejecución de GPU por primera vez, cargamos una serie de paquetes adicionales e inicializamos las estructuras y contextos de datos específicos de GPU.

In [ ]:
transactions.select(pl.col("AMOUNT").sum()).collect()

In [ ]:
transactions.select(pl.col("AMOUNT").sum()).collect()

We probably don't need a GPU for such a simple operation on this dataset. But when we start doing more complex analysis, the high-bandwidth memory and compute power of GPUs will make things much more interactive.

Probablemente no necesitemos una GPU para una operación tan simple en este conjunto de datos. Pero cuando comenzamos a hacer un análisis más complejo, la memoria de alto ancho de banda y el poder de calcular de las GPU hará que las cosas sean mucho más interactivas.

# More Complex Analysis

While the data is synthetic, it's representative of the kinds of datasets that come up in financial services, retail/e-commerce, consumer internet, and other industries.

With this data, we can see how using GPU-accelerated Polars provides a significant productivity boosts by exploring common business questions like:

1. Which customers have the largest total transctions?
2. What's the per-month transaction amount for each spending category over time?
3. How does weather effect the transactions for each category?

Because this data is synthentic, the results won't be meaningful in terms of business conclusions. Rather, we are motivating _why_ using this GPU engine is useful in terms of perfomance and cost reduction. We'll prove that the CPU and GPU engine get the same results in the first example, but as we move throughout the notebook we'll take this for granted and focus on CPU and GPU timings.

Let's start with our first question.

Si bien los datos son sintéticos, es representativo de los tipos de conjuntos de datos que surgen en servicios financieros, comercio minorista/e-comercio, Internet de consumo y otras industrias.

Con estos datos, podemos ver cómo el uso de polares acelerados con GPU proporciona un aumento significativo de la productividad al explorar preguntas comerciales comunes como:

¿Qué clientes tienen las trascciones totales más grandes?
¿Cuál es el monto de la transacción por mes para cada categoría de gastos con el tiempo?
¿Cómo afecta el clima las transacciones para cada categoría?
Debido a que estos datos son sinténticos, los resultados no serán significativos en términos de conclusiones comerciales. Más bien, estamos motivando por qué usar este motor GPU es útil en términos de perfomencia y reducción de costos. Probaremos que la CPU y el motor GPU obtienen los mismos resultados en el primer ejemplo, pero a medida que avanzamos a lo largo del cuaderno, daremos esto por sentado y nos enfocaremos en los tiempos de CPU y GPU.

Comencemos con nuestra primera pregunta.


## Which customers have the largest total transactions?

In [ ]:
%%time

res_cpu = (
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").sum())
    .sort(by="AMOUNT", descending=True)
    .head()
    .collect()
)
res_cpu

In [ ]:
%%time

res_gpu = (
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").sum())
    .sort(by="AMOUNT", descending=True)
    .head()
    .collect()
)
res_gpu

In [ ]:
assert_frame_equal(res_cpu, res_gpu)

Great! We see a nice performance gain when using the GPU engine and we've confirmed the results are equal.

In addition to the Dataframe interface, Polars also has an SQL interface. We can also use this with the GPU engine, since Polars translates both the DataFrame and SQL interfaces into a query execution plan.

¡Excelente! Vemos una buena ganancia de rendimiento al usar el motor GPU y hemos confirmado que los resultados son iguales.

Además de la interfaz DataFrame, Polars también tiene una interfaz SQL. También podemos usar esto con el motor GPU, ya que Polars traduce las interfaces DataFrame y SQL en un plan de ejecución de consultas.

¡Excelente! Vemos una buena ganancia de rendimiento al usar el motor GPU y hemos confirmado que los resultados son iguales.

Además de la interfaz DataFrame, Polars también tiene una interfaz SQL. También podemos usar esto con el motor GPU, ya que Polars traduce las interfaces DataFrame y SQL en un plan de ejecución de consultas.

¡Excelente! Vemos una Buena Ganancia de Rendimiento Al Usar El Motor Gpu y Hemos Confirmado Que Los Resultados Son IGuales.

Además de la Interfaz DataFrame, Polars También Tiene una Interfaz SQL. También Podemos USAR ESTO CON EL MOTOR GPU, YA QE POLARS CRODUCE LAS Interfaces DataFrame y Sql en Un Plan de Ejecución de Consultas.


In [ ]:
import polars as pl

query = """
SELECT CUST_ID, SUM(AMOUNT) as sum_amt
FROM transactions
GROUP BY CUST_ID
ORDER BY sum_amt desc
LIMIT 5
"""

# Try collecting without the GPU engine first
print(pl.sql(query).collect())

# If successful, try collecting with the GPU engine but disable optimizations
print(pl.sql(query).collect()) # Disable optimizations to see if it helps

In [ ]:
query = """
SELECT CUST_ID, SUM(AMOUNT) as sum_amt
FROM transactions
GROUP BY CUST_ID
ORDER BY sum_amt desc
LIMIT 5
"""

%time pl.sql(query).collect()
%time pl.sql(query).collect()

## Which customers have the largest single transaction?

Customer `CP2KXQSX9I` had the largest total transactions over time (on the full dataset), but they might not have the largest single transaction. Let's find the top customers by single transaction amount.

¿Qué clientes tienen la transacción individual más grande?
El cliente CP2KXQSX9I tuvo las transacciones totales más grandes con el tiempo (en el conjunto de datos completo), pero podrían no tener la transacción única más grande. Encontremos los mejores clientes por cantidad de transacción única.

In [ ]:
%%time

(
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").max().alias("max_amount"))
    .sort(by="max_amount", descending=True)
    .head()
    .collect()
)

In [ ]:
%%time

(
    transactions
    .group_by("CUST_ID")
    .agg(pl.col("AMOUNT").max().alias("max_amount"))
    .sort(by="max_amount", descending=True)
    .head()
    .collect())


Again, we see a nice speedup using the GPU engine.

## Investigating a specific customer

Let's explore the transactions of a single customer now. What was `CIP0I11MG2`'s largest transaction?

Nuevamente, vemos una buena aceleración usando el motor GPU.

Investigar un cliente específico
Exploremos las transacciones de un solo cliente ahora. ¿Cuál fue la transacción más grande de CIP0I11MG2?

In [ ]:
%%time

(
    transactions
    .filter(pl.col("CUST_ID") == "CIP0I11MG2")
    .select(pl.col("AMOUNT").max())
    .collect()
)

In [ ]:
!pip install altair==5.4.0

In [ ]:
import altair as alt

In [ ]:
import polars as pl
import plotly.express as px

# Assuming 'transactions' is your Polars DataFrame
filtered_data = transactions.filter(pl.col("CUST_ID") == "CIP0I11MG2")
max_amount = filtered_data.select(pl.col("AMOUNT").max()).collect()[0, 0]

# Create a new Polars DataFrame with the maximum amount repeated
histogram_data = pl.DataFrame({"AMOUNT": [max_amount] * 1000})  # Adjust the number of repetitions as needed

# Convert to Pandas DataFrame
pandas_df = histogram_data.to_pandas()

# Create the histogram using Plotly
fig = px.histogram(pandas_df, x="AMOUNT", nbins=20, title="Histogram of Maximum Amounts for CUST_ID 'CIP0I11MG2'")
fig.show()

In [ ]:
%%time

(
    transactions
    .filter(pl.col("CUST_ID") == "CIP0I11MG2")
    .select(pl.col("AMOUNT").max())
    .collect().alt.hist()
)

### Sidebar: Why was performance using the CPU and GPU engines similar for this query?

What we've seen so far is that, for many common queries, we see significant performance increases when we use the GPU. But for this last one, things were more similar.

Using Polars on a GPU isn't **always** a surefire winner in terms of speed compared to Polars on a CPU. For simple queries that aren't computationally heavy, like the last query, we're often limited by the speed that we can read results from disk.

Let's confirm this by profiling.

Barra lateral: ¿Por qué el rendimiento utilizaba los motores CPU y GPU similar para esta consulta?
Lo que hemos visto hasta ahora es que, para muchas consultas comunes, vemos aumentos significativos de rendimiento cuando usamos la GPU. Pero para este último, las cosas fueron más similares.

El uso de polares en una GPU no siempre es un ganador seguro en términos de velocidad en comparación con los polares en una CPU. Para consultas simples que no son computacionalmente pesadas, como la última consulta, a menudo estamos limitados por la velocidad que podemos leer los resultados del disco.

Confirmemos esto mediante el perfil.


In [ ]:
res, prof = (
    transactions
    .filter(pl.col("CUST_ID") == "CIP0I11MG2")
    .select(pl.col("AMOUNT").max())
    .profile()
)

prof.with_columns(
    ((pl.col("end") - pl.col("start")) / pl.col("end").max() * 100)
    .alias("pct_time_spent")
)

We spent 99.9%+ of the time just reading the data. Polars can use the GPU-accelerated Parquet reader to read this data, but ultimately when we're **IO bound** like this, there's less opportunity for GPU-acceleration.

Let's try an even more computationally intense query.

Gastamos más del 99.9%de las veces solo leyendo los datos. Los polares pueden usar el lector de parquet acelerado por GPU para leer estos datos, pero en última instancia, cuando estamos atados a IO así, hay menos oportunidades para la aceleración de GPU.

Intentemos una consulta aún más intensa computacionalmente.

## What's the per-month transaction amount for each category over time?

For this query, we'll group and sort down to the individual month, which takes more work.

¿Cuál es el monto de la transacción por mes para cada categoría con el tiempo?
Para esta consulta, agruparemos y clasificaremos al mes individual, lo que requiere más trabajo.

In [ ]:
%%time

res = (
    transactions
    .group_by(["EXP_TYPE", "YEAR", "MONTH"])
    .agg(pl.mean("AMOUNT"))
    .sort(["EXP_TYPE", "YEAR", "MONTH"])
    .collect()
)

In [ ]:
%%time

res = (
    transactions
    .group_by(["EXP_TYPE", "YEAR", "MONTH"])
    .agg(pl.mean("AMOUNT"))
    .sort(["EXP_TYPE", "YEAR", "MONTH"])
    .collect(engine=gpu_engine)
)

Again, since this query does more work we see strong performance benefits on the GPU.

## Interoperability

As mentioned in the introduction, all results will be materialized as standard CPU Polars DataFrames.

That means that any results from GPU-accelerated queries interoperate seamlessly with the rest of the polars ecosystem. For example, we can use Polars' built-in plotting functionality on the result we created from the GPU engine.

**Colab Note**: Using HoloViews/Bokeh in Colab is not fully supported, so plots may not render correctly.

Nuevamente, dado que esta consulta hace más trabajo, vemos fuertes beneficios de rendimiento en la GPU.

Interoperabilidad
Como se mencionó en la introducción, todos los resultados se materializarán como marcos de datos de Polars de CPU estándar.

Eso significa que cualquier resultado de consultas aceleradas por GPU interopere a la perfección con el resto del ecosistema Polars. Por ejemplo, podemos usar la funcionalidad de trazado incorporada de Polars en el resultado que creamos desde el motor GPU.

Nota de Colab: El uso de Holoviews/Bokeh en Colab no es completamente compatible, por lo que las tramas pueden no rendir correctamente.


In [ ]:
res = res.with_columns(
    pl.datetime(pl.col("YEAR"), pl.col("MONTH"), day=1)
    .alias("year-mon")
)

res.hvplot.scatter(x="year-mon", y="AMOUNT", by="EXP_TYPE")

In [ ]:
res

Interoperability goes in the other direction, too. We can use materialized CPU DataFrames in downstream operations on the GPU. Polars will handle any necessary CPU/GPU conversions for us, which we'll see shortly.

# Moving Beyond Our Single Dataset

Let's bring in some weather data to evaluate if transactions are impacted by weather. For example, spending behavior for certain categories could be influenced by whether it's raining or even _how much_ it's raining.

Like our transactions dataset, here we'll use a simulated rainfall dataset.

## Is more rain related to more transactions across specific categories?

Our rainfall dataset is a CSV file, so we'll use the CSV reader here and take a look at a few rows.

In this case, we'll load the data using the CPU to highlight how smoothly Polars handles mixing CPU and GPU objects.

La interoperabilidad también va en la otra dirección. Podemos usar los marcos de datos de CPU materializados en las operaciones aguas abajo en la GPU. Los polares manejarán las conversiones de CPU/GPU necesarias para nosotros, que veremos en breve.

Ir más allá de nuestro conjunto de datos único
Traemos algunos datos meteorológicos para evaluar si las transacciones se ven afectadas por el clima. Por ejemplo, el comportamiento del gasto para ciertas categorías podría verse influenciado por si lloven o incluso cuánto está lloviendo.

Al igual que nuestro conjunto de datos de transacciones, aquí usaremos un conjunto de datos de lluvia simulada.

¿Está más lluvia relacionada con más transacciones en categorías específicas?
Nuestro conjunto de datos de lluvia es un archivo CSV, por lo que usaremos el lector CSV aquí y echaremos un vistazo a algunas filas.

En este caso, cargaremos los datos utilizando la CPU para resaltar cómo los polares sin problemas manejan los objetos de CPU y GPU.


In [ ]:
names = ['Location', 'Rainfall (inches)', 'Date', 'YEAR', 'MONTH', 'DAY']

weather = pl.scan_csv("rainfall_data_2010_2020.csv", new_columns=names)
weather.head().collect()

Data is often messy. We've got dates in this CSV file, but they weren't parsed as dates by the CSV reader due to the format. We can easily clean this up by typecasting, after which we'll materialize the DataFrame in CPU memory.

Los datos a menudo son complicados. Tenemos fechas en este archivo CSV, pero el lector de CSV no las analizó como fechas debido al formato. Podemos limpiarlo fácilmente esto mediante Typecasting, después de lo cual materializaremos el marco de datos en la memoria de la CPU.

In [ ]:
weather_cleaned = (
    weather
    .with_columns(pl.col("Date").cast(pl.Utf8).str.strptime(pl.Date(), "%Y%m%d"))
    .collect()
)

To answer the original question about rainfall and spending, we need to join these datasets together.

We can pass this CPU in-memory DataFrame to a Polars Lazy API query by calling `.lazy()`.

Para responder a la pregunta original sobre lluvia y gastos, necesitamos unir estos conjuntos de datos juntos.

Podemos pasar esta CPU en la memoria de datos a una consulta de la API Lazy de Polars llamando a .lazy ().

In [ ]:
%%time

(
    transactions
    .join(
        other=weather_cleaned.lazy(),
        left_on="DATE",
        right_on="Date",
        how="inner"
    )
    .group_by(["EXP_TYPE", "DATE"])
    .agg(pl.mean("Rainfall (inches)"))
    .sort(["DATE", "EXP_TYPE", "Rainfall (inches)"])
    .head()
    .collect()
)

When we run the same query on the GPU, Polars handles the CPU / GPU transfers for us under the hood.

Cuando ejecutamos la misma consulta en la GPU, Polars maneja las transferencias CPU / GPU para nosotros debajo del capó.

In [ ]:
%%time

(
    transactions
    .join(
        other=weather_cleaned.lazy(),
        left_on="DATE",
        right_on="Date",
        how="inner"
    )
    .group_by(["EXP_TYPE", "DATE"])
    .agg(pl.mean("Rainfall (inches)"))
    .sort(["DATE", "EXP_TYPE", "Rainfall (inches)"])
    .head()
    .collect(engine=gpu_engine)
)

Let's demonstrate some more of the native plotting facilities in Polars by answering the question in more holistic fashion.

We'll make a plot of total rainfall and total amount spent per month for each year and each type of expenditure.

Demostremos algunas más de las instalaciones de trama nativas en polares respondiendo la pregunta de manera más holística.

Haremos una parcela de lluvia total y la cantidad total gastada por mes por cada año y cada tipo de gastos.

In [ ]:
res = (
    transactions
    .join(
        other=weather_cleaned.lazy(),
        left_on="DATE",
        right_on="Date",
        how="inner"
    )
    .group_by(["EXP_TYPE", "YEAR", "MONTH"])
    .agg(pl.sum("Rainfall (inches)"), pl.sum("AMOUNT"))
    .sort(["YEAR", "MONTH"])
    .collect(engine=gpu_engine)
)

res

In [ ]:
(
    res
    .with_columns(
        pl.date(pl.col("YEAR"), pl.col("MONTH"), 1).alias("date-month"),
        pl.col("Rainfall (inches)")*100,
    )
    .hvplot.line(
        x="date-month", y=["AMOUNT", "Rainfall (inches)"],
        by=['EXP_TYPE'],
        rot=45,
    )
)

We intially constructed the GPU engine with `raise_on_fail=True` to demonstrate the breadth of GPU support. Now, we'll demonstrate how a not yet GPU supported feature is handled gracefully when we set `raise_on_fail=False` (the default behavior for `engine="gpu"`).

When we use `rolling_mean`, our query smoothly falls back to the CPU to continue running rather than failing.

Construimos intencionalmente el motor GPU con Raise_on_Fail = True para demostrar la amplitud del soporte de GPU. Ahora, demostraremos cómo una característica compatible con GPU aún no se maneja con gracia cuando establecemos Rais_on_Fail = False (el comportamiento predeterminado para Engine = "GPU").

Cuando usamos Rolling_mean, nuestra consulta recae suavemente a la CPU para continuar funcionando en lugar de fallar.

In [ ]:
engine_with_fallback = pl.GPUEngine(
    device=0, # This is the default
    raise_on_fail=False, # Fallback to CPU if we can't run on the GPU (this is the default)
)

In [ ]:
%%time

result = transactions.with_columns(
    pl.col('AMOUNT').rolling_mean(
        window_size=7
    )
     .alias('rolling_avg')
).collect(engine=engine_with_fallback)

result.head()

# Conclusion

With the new Polars GPU engine powered by cuDF, you can potentially get significant performance gains and lower costs for workflows processing 10-100+ GB of data.

To learn more, we encourage you to visit the Polars GPU Support documentation or visit rapids.ai/cudf-polars.

Con el nuevo motor de GPU Polars alimentado por CUDF, puede obtener ganancias de rendimiento significativas y menores costos para los flujos de trabajo que procesan 10-100+ GB de datos.

Para obtener más información, le recomendamos que visite la documentación de soporte de GPU de Polars o visite Rapids.ai/cudf-polars.